In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

UsageError: Line magic function `%` not found.


In [52]:
dataset = pd.read_csv('spam.csv')

In [53]:
dataset.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [54]:
dataset.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [55]:
dataset.shape

(5572, 2)

In [56]:
# You can see that there are high number of spam's in the dataset
# So, let's do one hot encoding in category label
category_col = pd.get_dummies(dataset['Category'])
category_col.head()

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [57]:
# Now lets add this spam and ham columns to main dataset
dataset = pd.concat([dataset,category_col],axis='columns')
dataset

,Category,Message,ham,spam
0,ham,"Go until jurong point, crazy.. Available only ...",1,0
1,ham,Ok lar... Joking wif u oni...,1,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,ham,U dun say so early hor... U c already then say...,1,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",1,0
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,0,1
5568,ham,Will ü b going to esplanade fr home?,1,0
5569,ham,"Pity, * was in mood for that. So...any other s...",1,0
5570,ham,The guy did some bitching but I acted like i'd...,1,0


In [58]:
# Now lets drop the category and ham column
dataset.drop(['Category','ham'],axis='columns',inplace=True)
dataset.head()

,Message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [59]:
# Now lets train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.Message,dataset.spam,test_size = 0.25)

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [61]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [62]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=uint8)

In [63]:
# Accirding to the model, the 2nd email is a Spam.

In [64]:
X_test_count = v.transform(X_test)
model.score(X_test_count, y_test)

0.9842067480258435

#### SKLEARN PIPELINE

In [65]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [66]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [67]:
clf.score(X_test,y_test)

0.9842067480258435

In [68]:
clf.predict(emails)

array([0, 1], dtype=uint8)